In [1]:
import pandas as pd
import geopandas
import json
import folium
from shapely import wkt

<h1>Przygotowanie ulic</h1>

In [2]:
with open("ulice") as file:    
    streets = json.load(file)
streetsDF = pd.DataFrame(streets["Data"]["Table"])
streetsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   332 non-null    int64 
 1   Name                 332 non-null    object
 2   Wkt                  332 non-null    object
 3   AreaId               332 non-null    int64 
 4   AreaName             332 non-null    object
 5   OccupancyRate        332 non-null    int64 
 6   OccupancyRateDate    332 non-null    object
 7   OccupancyRateName    332 non-null    object
 8   OccupancyRateColor   332 non-null    int64 
 9   OccupancyRateSource  332 non-null    object
 10  PlacesCount          332 non-null    int64 
dtypes: int64(5), object(6)
memory usage: 28.7+ KB


In [3]:
streetsGDF = geopandas.GeoDataFrame(streetsDF[streetsDF["Wkt"] != ""])
streetsGDF["geometry"] = streetsGDF["Wkt"].apply(wkt.loads)

<h2>Przygotowanie ulic ze względu na zajętość</h2>

In [4]:
streetsLackPlaces = streetsGDF[streetsGDF["OccupancyRateName"] == "BRAK"]
streetsLackPlaces = geopandas.GeoDataFrame(streetsLackPlaces, geometry="geometry", crs = 4326)

streetsLowNumPlaces = streetsGDF[streetsGDF["OccupancyRateName"] == "MAŁO"]
streetsLowNumPlaces = geopandas.GeoDataFrame(streetsLowNumPlaces, geometry="geometry", crs = 4326)

streetsHighNumPlaces = streetsGDF[streetsGDF["OccupancyRateName"] == "DUŻO"]
streetsHighNumPlaces = geopandas.GeoDataFrame(streetsHighNumPlaces, geometry="geometry", crs = 4326)

<h2>Przygotowanie ulic ze względu na strefy</h2>

In [5]:
zoneBlue = streetsGDF[streetsGDF["AreaId"] == 2]
zoneBlue["Geometry"] = zoneBlue[zoneBlue["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneBlue.drop("Wkt", axis=1, inplace=True)
zoneBlue = geopandas.GeoDataFrame(zoneBlue, geometry="Geometry", crs = 4326)

zoneOrange = streetsGDF[streetsGDF["AreaId"] == 3]
zoneOrange["Geometry"] = zoneOrange[zoneOrange["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneOrange.drop("Wkt", axis=1, inplace=True)
zoneOrange = geopandas.GeoDataFrame(zoneOrange, geometry="Geometry", crs = 4326)

zoneFreeInside = streetsGDF[streetsGDF["AreaId"] == 4]
zoneFreeInside["Geometry"] = zoneFreeInside[zoneFreeInside["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneFreeInside.drop("Wkt", axis=1, inplace=True)
zoneFreeInside = geopandas.GeoDataFrame(zoneFreeInside, geometry="Geometry", crs = 4326)

zoneFreeOutside = streetsGDF[streetsGDF["AreaId"] == 5]
zoneFreeOutside["Geometry"] = zoneFreeOutside[zoneFreeOutside["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneFreeOutside.drop("Wkt", axis=1, inplace=True)
zoneFreeOutside = geopandas.GeoDataFrame(zoneFreeOutside, geometry="Geometry", crs = 4326)

zoneGreen = streetsGDF[streetsGDF["AreaId"] == 6]
zoneGreen["Geometry"] = zoneGreen[zoneGreen["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneGreen.drop("Wkt", axis=1, inplace=True)
zoneGreen = geopandas.GeoDataFrame(zoneGreen, geometry="Geometry", crs = 4326)

zoneRed = streetsGDF[streetsGDF["AreaId"] == 7]
zoneRed["Geometry"] = zoneRed[zoneRed["Wkt"]!=""]["Wkt"].apply(wkt.loads)
zoneRed.drop("Wkt", axis=1, inplace=True)
zoneRed = geopandas.GeoDataFrame(zoneRed, geometry="Geometry", crs = 4326)

M:\Programs\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mbiel\AppData\Local\Temp\ipykernel_2252\4155955117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zoneBlue.drop("Wkt", axis=1, inplace=True)
M:\Programs\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

<h1>Przygotowanie tablic informacyjnych</h1>

In [6]:
with open("tablice") as file:    
    tables = json.load(file)
tablesDF = pd.DataFrame(tables["Data"]["Tables"])
tablesDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   TableId    12 non-null     int64 
 1   TableName  12 non-null     object
 2   GPS        12 non-null     object
 3   Items      12 non-null     object
dtypes: int64(1), object(3)
memory usage: 512.0+ bytes


In [7]:
for index, row in tablesDF.iterrows():
    coordinates = row["GPS"].split(";")
    tablesDF.loc[index, "Longitude"] = [float(coordinates[1])]
    tablesDF.loc[index, "Latitude"] = [float(coordinates[0])]
    tablesDF["Geometry"] = geopandas.points_from_xy(tablesDF["Longitude"], tablesDF["Latitude"], crs="EPSG:4326")
tablesDF.crs = {"init": "EPSG:4326"}
tablesDF = geopandas.GeoDataFrame(tablesDF, geometry="Geometry", crs = 4326)

C:\Users\mbiel\AppData\Local\Temp\ipykernel_2252\443221028.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tablesDF.crs = {"init": "EPSG:4326"}


<h1>Przygotowanie parkomatów (plik XLSX pobrany ze strony Rzeszowa)</h1>

In [8]:
parkingMeters = pd.read_excel("Lokalizacja parkomatów w Strefie Płatnego Parkowania w Rzeszowie.XLSX")
parkingMeters = geopandas.GeoDataFrame(parkingMeters)
parkingMeters["geometry"] = geopandas.points_from_xy(parkingMeters["DŁUGOŚĆ GEOGRAFICZNA"], parkingMeters["SZEROKOŚĆ GEOGRAFICZNA"], crs="EPSG:4326")

<h1>Wyświetlenie mapy z zajętością</h1>

In [9]:
mapCapacity = folium.Map(location=[50.03, 22.00], zoom_start=12)

<h3>Parkometry</h3>

In [10]:
for index, parkingMeter in parkingMeters.iterrows():
    folium.Marker([parkingMeter["SZEROKOŚĆ GEOGRAFICZNA"], parkingMeter["DŁUGOŚĆ GEOGRAFICZNA"]]).add_to(mapCapacity)

<h3>Tablice</h3>

In [11]:
for index, table in tablesDF.iterrows():
    folium.Marker(location=[table["Latitude"], table["Longitude"]], icon = folium.Icon(color="black")).add_to(mapCapacity)

<h3>Ulice</h3>

In [12]:
folium.Choropleth(
    streetsLackPlaces["geometry"],
    line_weight=3,
    line_color='red'
).add_to(mapCapacity)

folium.Choropleth(
    streetsLowNumPlaces["geometry"],
    line_weight=3,
    line_color='orange'
).add_to(mapCapacity)

folium.Choropleth(
    streetsHighNumPlaces["geometry"],
    line_weight=3,
    line_color='green'
).add_to(mapCapacity)

<h2>Mapa z zajętością</h2>

In [13]:
mapCapacity

<h1>Wyświetlenie mapy z podziałem na strefy</h1>

In [14]:
mapZones = folium.Map(location=[50.03, 22.00], zoom_start=12)

<h3>Parkometry</h3>

In [15]:
for index, parkingMeter in parkingMeters.iterrows():
    folium.Marker([parkingMeter["SZEROKOŚĆ GEOGRAFICZNA"], parkingMeter["DŁUGOŚĆ GEOGRAFICZNA"]],
              icon = folium.Icon(color="black")
              ).add_to(mapZones)

<h3>Tablice</h3>

In [16]:
for index, table in tablesDF.iterrows():
    folium.CircleMarker(location=[table["Latitude"], table["Longitude"]], color="black", radius=15).add_to(mapZones)

<h3>Ulice</h3>

In [17]:
folium.Choropleth(
    zoneBlue["Geometry"],
    line_weight=3,
    line_color='blue'
).add_to(mapZones)

folium.Choropleth(
    zoneOrange["Geometry"],
    line_weight=3,
    line_color='orange'
).add_to(mapZones)

folium.Choropleth(
    zoneFreeInside["Geometry"],
    line_weight=3,
    line_color='purple'
).add_to(mapZones)

folium.Choropleth(
    zoneFreeOutside["Geometry"],
    line_weight=3,
    line_color='gray'
).add_to(mapZones)

<h2>Mapa z podziałem na strefy</h2>

In [18]:
mapZones